In [2]:
import json
import numpy as np
import pandas as pd
import cv2
import os
import scipy.misc
import random

from PIL import Image
from sklearn import preprocessing
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats, integrate, misc
from skimage import io,img_as_float

%matplotlib inline

In [3]:
# GLOBAL CONSTANTS
DATA_FILE = 'leafsnap-dataset-images.csv'
NUM_CLASSES = 185

In [4]:
columns = ['file_id', 'image_pat', 'segmented_path', 'species', 'source']
data = pd.read_csv(DATA_FILE, names=columns, header=1)
display(data.head())

images     = data['image_pat']
images_seg = data['segmented_path']
species    = data['species']
species_classes = sorted(set(species))
print('Number of Samples: {}'.format(len(images)))

,file_id,image_pat,segmented_path,species,source
0,55498,dataset/images/lab/abies_concolor/ny1157-01-2.jpg,dataset/segmented/lab/abies_concolor/ny1157-01...,Abies concolor,lab
1,55499,dataset/images/lab/abies_concolor/ny1157-01-3.jpg,dataset/segmented/lab/abies_concolor/ny1157-01...,Abies concolor,lab
2,55500,dataset/images/lab/abies_concolor/ny1157-01-4.jpg,dataset/segmented/lab/abies_concolor/ny1157-01...,Abies concolor,lab
3,55501,dataset/images/lab/abies_concolor/ny1157-02-1.jpg,dataset/segmented/lab/abies_concolor/ny1157-02...,Abies concolor,lab
4,55502,dataset/images/lab/abies_concolor/ny1157-02-2.jpg,dataset/segmented/lab/abies_concolor/ny1157-02...,Abies concolor,lab


Number of Samples: 30865


In [5]:
def load_image_and_preprocess(path, segmented_path, flip_left_right=False, flip_up_down=False, rotate_180=False):
    # Open image from disk and flip it if generating data.
    image = misc.imread(path.strip())
    segmented_image = misc.imread(segmented_path.strip())
    
    img = segmented_image
    h, w = img.shape[:2]
    height,width = h,w
    # print('Height: {:3d}, Width: {:4d}\n'.format(height,width))
    ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # Calculate bounding rectangles for each contour.
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    if rects == []:
        # No segmentation in image
        return []
    else:
        #Calculate the combined bounding rectangle points.
        top_y = max(0, min([y for (x, y, w, h) in rects]) - 40)
        bottom_y = min(height, max([y+h for (x, y, w, h) in rects]) + 80)
        left_x = max(0, min([x for (x, y, w, h) in rects]) - 40)
        right_x = min(width-1, max([x+w for (x, y, w, h) in rects]) + 80)
        if left_x >= 455 or top_y >= 480:
            return []

#    print('[INFO] Bounding Box:\nLeft X: {:3d},  Right X: {:d}\nTop Y : {:3d}, Bottom Y: {:3d}'.format(left_x, right_x, top_y, bottom_y))

    # Use the rectangle to crop on original image
    img = image[top_y:bottom_y, left_x:right_x]
    img = scipy.misc.imresize(img, (264,264))
    return img

In [6]:
# sample = random.sample(range(0, len(images)), 3)
# sample = [6320, 23424, 25187]
print('[INFO] Processing Images')
cropped_images = []
image_species = []
image_paths = []
count = 0
for index in range(len(images)):
    image = load_image_and_preprocess(images[index], images_seg[index])
    if type(image) != type([]):
        cropped_images.append(image)
        image_species.append(species[index])
        file_name = 'cropped_images/{}/{}.jpg'.format(species[index].title(), count)
        print(file_name)
        image_paths.append(file_name)
        cv2.imwrite(file_name, image)
        count += 1
    if index > 0 and index%1000 == 0:
        print('[INFO] Processed {:5d} images'.format(index))
#         _ = plt.imshow(image)
#         plt.title('Species: {}\nImage Path: {}\nSegmented Path: {}\n'.format(species[index], images[index], images_seg[index]))
#         plt.show()
#    else:
#         print('[ERROR] Image has no segmentation!')
#         print('Image Path: {}\n'.format(images[index]))
print('[DONE]')

[INFO] Processing Images
cropped_images/Abies Concolor/0.jpg
cropped_images/Abies Concolor/1.jpg
cropped_images/Abies Concolor/2.jpg
cropped_images/Abies Concolor/3.jpg
cropped_images/Abies Concolor/4.jpg
cropped_images/Abies Concolor/5.jpg
cropped_images/Abies Concolor/6.jpg
cropped_images/Abies Concolor/7.jpg
cropped_images/Abies Concolor/8.jpg
cropped_images/Abies Concolor/9.jpg
cropped_images/Abies Concolor/10.jpg
cropped_images/Abies Concolor/11.jpg


KeyboardInterrupt: 